In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

import re
import xml.etree.ElementTree as ET
import pandas as pd

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
from pylib import *
pkg

`isr.umich.edu-psid-inequality-1.1.1` Last Update: 2023-06-06T04:38:40

__



## Updating Data

To update the data, use the data extration tool at the (PSID website. )[https://simba.isr.umich.edu/Cart/co.aspx], adding any new variables to your data cart. The (variable entry text box)[https://simba.isr.umich.edu/DC/l.aspx] is a handy way to copy the existing varibles into a new cart.  When you checkout your data cart, ( click on the cart icon ) fetch your dataset with these options: 

* Codebook Type: XML
* Data output type: dBase Data File, create compressed file

Download the ZIP file ( you might have to recompress it if your browser unpacks it ) and move it into the ``data`` directory. Set the reference to the new ZIP file, then re-build the package.
## Documentation Links

* [PSID Homepage](https://psidonline.isr.umich.edu/default.aspx) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[psid_ineq](notebooks/extract.ipynb#df)**. PSID, Inequality extract
* **[_labels](notebooks/extract.ipynb#labels)**. Categorical value labels
* **[data_dictionary](notebooks/extract.ipynb#mdf)**. List of Variables

## References
<ul><li> <strong><a href="https://psidonline.isr.umich.edu/help/xyr/psid.xlsx">variables</a></strong>. Variable descriptions, in a cross-year format.</li><li> <strong>source</string>, <em>data/J321027.zip</em>. Downloaded source file</li><ul>

In [3]:
# create URLs to the files inside the data distribution ZIP file
surl   = pkg.reference('source').resolved_url
base_name = surl.resource_file.split('.')[0]
database_url = surl.set_target_file(base_name+'.dbf')
codebook_url = surl.set_target_file(base_name+'_codebook.xml')

codebook_url

<ZipUrl file:///Users/eric/proj/data-projects/metatab-packages/umich.edu/isr.umich.edu-psid-inequality/data/J321027.zip#J321027_codebook.xml>

In [4]:
from simpledbf import Dbf5
dbf = Dbf5(database_url.get_target().fspath)
df = dbf.to_dataframe()

# Make everything as numerical as possible
for c in df.columns:
    df[c] = pd.to_numeric(df[c])


PyTables is not installed. No support for HDF output.


In [5]:

# Recommended construction of unique persion id
df['pid'] = df['ER30001'] + df['ER30002']

# opening_columns. These should always be at the start of the schema. 
init_cols = ['ER30000','ER30001','ER30002']


/var/folders/kp/jn04x74j43j9v0t6cl0574h00000gn/T/ipykernel_49353/1716784282.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['pid'] = df['ER30001'] + df['ER30002']


In [6]:
# The Path has some useful information about the variable.
varis = pkg.reference('variables').dataframe()

t = varis.set_index(['TYPE', 'CATEGORY', 'TEXT', 'HEAD_WIFE', 'VAR_COUNT'])
t = t.stack().to_frame('NAME')

t = t.reset_index()
t.columns = ['type','category','text','head_wife','var_count','year','name']

varis = t

varis['path'] = varis.text.apply(clean_text)
varis['year'] = varis.year.apply(lambda v: int(v[1:]) )

# XML Codebook is the main source of variable information

mdf = xml_to_dataframe(codebook_url.get_target().fspath)
mdf['YEAR'] = pd.to_numeric(mdf.YEAR)
mdf.columns = [c.lower() for c in mdf.columns]
t = mdf.merge(varis, on= ('year','name' ))

t['qtext'] = t.qtext.apply(lambda v: v.replace('\n',' ') )
t['etext'] = t.etext.apply(lambda v: v.replace('\n',' ') )

mdf = t
mdf.head()

,year,type_id,name,label,qtext,etext,type,category,text,head_wife,var_count,path
0,1968,0,ER30000,RELEASE NUMBER,Release Number,For Release 2 we have updated Relation to Hea...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,/survey information:/interview information/rel...
1,1968,0,ER30001,1968 INTERVIEW NUMBER,1968 Interview Number (1968 ID Number),This variable is the 1968 family ID number. T...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,/survey information:/interview information/id ...
2,1968,0,ER30002,PERSON NUMBER 68,Person Number,,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,/survey information:/interview information/per...
3,2001,1,ER17001,RELEASE NUMBER,Release Number,"May 2008: Variables ER20395-ER20458, formerly...",FAMILY PUBLIC,SURVEY INFORMATION,01>SURVEY INFORMATION\n02>Interview Informatio...,,16.0,/survey information/interview information/rele...
4,2001,1,ER17012,# IN FU,Number of Persons in FU at the Time of the 200...,This variable represents the actual number of ...,FAMILY PUBLIC,FAMILY COMPOSITION,01>FAMILY COMPOSITION\n02>Current:\n03>number ...,,41.0,/family composition/current:/number in family ...


In [7]:
# Re-order the data dictionary
mdf_sel = mdf[mdf.name.isin(df.columns)].copy()

mdf_sel = mdf_sel.sort_values(['category', 'label', 'year'])

is_init = mdf_sel['name'].isin(init_cols)
is_si = mdf_sel.category == 'SURVEY INFORMATION'

a = mdf_sel.loc[ is_init ]
b = mdf_sel.loc[~is_init &  is_si]
c = mdf_sel.loc[~is_init & ~is_si]

mdf_sel = pd.concat([a,b,c])

mdf_sel

,year,type_id,name,label,qtext,etext,type,category,text,head_wife,var_count,path
1,1968,0,ER30001,1968 INTERVIEW NUMBER,1968 Interview Number (1968 ID Number),This variable is the 1968 family ID number. T...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,/survey information:/interview information/id ...
2,1968,0,ER30002,PERSON NUMBER 68,Person Number,,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,/survey information:/interview information/per...
0,1968,0,ER30000,RELEASE NUMBER,Release Number,For Release 2 we have updated Relation to Hea...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,1.0,/survey information:/interview information/rel...
56,2001,2,ER33601,2001 INTERVIEW NUMBER,2001 Interview Number (2001 ID Number),The values for this variable represent the 200...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,/survey information:/interview information/id ...
119,2003,2,ER33701,2003 INTERVIEW NUMBER,2003 Interview Number (2003 ID Number),The values for this variable represent the 200...,INDIVIDUAL,SURVEY INFORMATION,01>SURVEY INFORMATION:\n02>Interview Informati...,,41.0,/survey information:/interview information/id ...
...,...,...,...,...,...,...,...,...,...,...,...,...
530,2015,1,ER64873,L72 OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of (your/his/her) first full-tim...,The 3-digit occupation code from 2000 CENSUS ...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."
177,2005,1,ER27446,L72-72A OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of your first full-time regular ...,See the note at ER27352. The 3-digit occupati...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."
246,2007,1,ER40618,L72-72A OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of your first full-time regular ...,See the note at ER40527. The 3-digit occupati...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."
315,2009,1,ER46596,L72-72A OCCUPATION 1ST FULL TIME JOB-HD,L72. Thinking of your first full-time regular ...,See the note at ER46504. The 3-digit occupati...,FAMILY PUBLIC,WORK,01>WORK\n02>Occupation\n03>1st full-time job\n...,HEAD,7.0,"/work/occupation/st full-time job/-digit, cen..."


In [8]:
pkg = mp.jupyter.open_source_package()
ss = pkg['Schema']
ss.args = ['DataType',  'Year', 'Label', 'Category', 'Description', 'Has_codes', 'Labels']

st = pkg.resource('psid_ineq').schema_term

# Clears out all of the schema children.


for c in st.children:
    ss.remove_term(c)
st.children = []

final_cols = ['pid'] + list(mdf_sel.name)

st.new_child('Table.Column','pid', DataType='integer',Description='Constructed person id')

for idx, c in mdf_sel.iterrows():
    t = st.new_child('Table.Column', c['name'].lower())
    t['DataType'] = 'number' if df[c['name']].dtype == 'float64' else 'integer'
    t['Year'] = c.year
    t['Label'] = c.label
    t['Category'] = c.category
    t['Description'] = (c.qtext+": "+c.etext).strip()

pkg.write()


# Make df match

df = df[final_cols]


In [9]:
labels = extract_code_labels(codebook_path)
labels = labels.rename(columns={'name':'column','value':'code'})
labels['column'] = labels['column'].str.lower()

rows = []
for idx, r in labels.iterrows():
    code = clean_code(r.code)

    if isinstance(code, list):
        
        r['code'] = np.nan
        r['low'], r['high'] = code
    elif isinstance(code, int):
        r['code'] = code
    else:
        assert False # Looks like this never actually happens
        r['value'] = code
        
    rows.append(r)
    
labels = pd.DataFrame(rows)
labels = labels[['column','code', 'low','high','label']]

#
# for columns where there is a mix of codes and high/low, move everything
# over to value/high/low
# 

mixed_cols = []

for gn, g in labels.groupby('column'):
    
    # Either all of the codes are Nan, or all of the High/Low are nan, 
    # but there should never be some of each
    
    code_nans = g.code.isnull().sum()
    low_nans = g.low.isnull().sum()
    high_nans = g.high.isnull().sum()
    
    assert low_nans == high_nans
    
    if code_nans > 0 and low_nans > 0:
        mixed_cols.append(gn)
        
mc_idx = labels.column.isin(mixed_cols)& (~labels.code.isnull())
    

labels.loc[mc_idx,'low' ] = labels.code
labels.loc[mc_idx,'high' ] = labels.code   
labels.loc[mc_idx,'code' ] = np.nan

labels.sample(10)



NameError: name 'codebook_path' is not defined

In [ ]:
mc_idx = (labels.low == labels.high)
cols = labels.loc[mc_idx].column.unique()
labels[labels.column.isin(cols)].sample(10)

In [ ]:
mdf_map = { r['name']:r for idx, r in mdf.iterrows() }
mdf.head().T